In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
fname = "train_fold_one"
basepath = "./-paraphrase-multilingual-MiniLM-L12-v2_3015"

## Load data

In [3]:
import pandas as pd

def loadData(fname, basepath):
    sample = pd.read_csv(f"{basepath}/{fname}.csv")
    sample['Document'] = sample.apply(lambda row: ': '.join(row.astype(str)), axis=1)
    return sample

## Question Generation

In [4]:
from qa_generation import QAGenerator
from prompt_factory import PromptFactory
from tqdm.notebook import tqdm

template = '''
        Gedraag je als een {role} die online informatie zoekt.
        Patiënten stellen meestal vragen als:
{{
    "qa_list": [
        {{
            "question": "Hoe kan ik een verstopte voedingssonde doorspoelen?",
            "answer": "Om een verstopte voedingssonde door te spoelen, kunt u de volgende stappen proberen: 1. Sluit de spuit rechtstreeks aan op de sonde (niet op het voedingssysteem). Als er een verstopping is in het voedingssysteem, kunt u dit vervangen. 2. Neem een spuit van 10 cc en spuit met lichte druk lauwwarm water door de voedingssonde. Herhaal dit zo nodig nog een keer. 3. Als het oplossen van de verstopping niet lukt, laat dan lauwwarm water 30 minuten inwerken en herhaal de bovenstaande procedure nogmaals. 4. Als u de verstopping kunt zien, kunt u proberen om de voedingssonde op die plek zachtjes te kneden. Als de verstopping dan loskomt, kunt u het doorspuiten met lauwwarm water. Belangrijk: Gebruik nooit een voerdraad of koolzuurhoudend bronwater of frisdranken, omdat dit kan leiden tot perforatie. Daarnaast wordt het afraden om natriumbicarbonaat te gebruiken als medicatie de oorzaak is van de verstopping, omdat dit de verstopping groter kan maken."
        }}
    ]
}}


        in het json formaat: 
        {format}

Document Informatie:
        {document}

        Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen zoals 'dit onderzoek' of 'die procedure'. Verwijs direct naar de procedure of het document met de exacte naam om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden en zorg ervoor dat elke vraag direct gerelateerd is aan de verstrekte documentinformatie.


        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
        {format}

        Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

roles = [
    'Patiënt',
    'Nieuwe Ouder',
    'Oudere Patiënt',
    'Persoon die een Tweede Mening Zoekt',
    'Reiziger die Medisch Advies Na Reizen Zoekt',
    'Zorgverlener die Informatie Zoekt']

prompt_factory = PromptFactory(prompt=template,roles=roles)

num_questions_per_doc = 5

QAGenerator = QAGenerator()
# Generate the data
import pandas as pd
from tqdm import tqdm


def generateQuestions(df):
    data = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0],desc="Generating QAs"):
        doc = row["Document"]
        # Do something with col1 and col2
        try:
            for i in range(2):
                qa_list = QAGenerator.generate_qas(prompt_factory.generate_prompt(doc, num_questions_per_doc))
                for qa in qa_list:
                    new_row = row.to_dict()
                    new_row['Question'] = qa.question
                    new_row['Answer'] = qa.answer
                    data.append(new_row)

        except Exception as e:
                print(f"Failed to generate QA for doc: {doc[:100]}. Error: {e}")
                continue
    if(len(data) == 0):
       return pd.DataFrame()
    result = pd.DataFrame(data)
    result = result.dropna(subset=['Document', 'Question', 'Answer'])
    result = result[(result['Document'] != '') & (result['Question'] != '') & (result['Answer'] != '')]
    return result


In [5]:
from fact_qa import FactQAGenerator
from prompt_factory import PromptFactory
import langchain

template = '''
Gedraag je als iemand die online informatie zoekt en stelt vragen gebaseerd op specifieke feiten zoals locatie, telefoonnummer, naam, enzovoort.

Patiënten of gebruikers kunnen bijvoorbeeld vragen stellen als:
{{
    "qa_list": [
        {{
            "question": "Wat is het telefoonnummer van de kliniek voor noodgevallen?",
            "answer": "Het telefoonnummer van de kliniek voor noodgevallen is 012-345-6789."
        }},
        {{
            "question": "Waar is de hoofdingang van het ziekenhuis gelegen?",
            "answer": "De hoofdingang van het ziekenhuis is gelegen aan de Eerste Gezondheidsstraat 123 in Amsterdam."
        }}
        // meer vragen en antwoorden gebaseerd op specifieke feiten
    ]
}}

in het json formaat:
{format}

Document Informatie:
{document}

Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen. Verwijs direct naar de specifieke feit (zoals locatie, telefoonnummer, naam) om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden.

Maak vragen die beantwoord kunnen worden met de volgende informatie: {fact}

Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
{format}

Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

base_prompt = langchain.PromptTemplate(
            template=template,
            input_variables=['fact', 'n', 'document', 'format']
        )

num_questions_per_doc = 5

FactQAGenerator = FactQAGenerator()

def generateFactQuestions(df):
    data = []
    # Generate the data
    for index, row in tqdm(df.iterrows(), total=df.shape[0],desc="Generating Fact QAs"):
        doc = row['Document']
        # Assuming phone numbers are separated by commas
        facts = row['Context Entities']
        try:
            for i in range(2):
                for entity in extract_desired_entities(facts)[:5]:
                    fact = entity["value"]
                    prompt = base_prompt.partial(
                        n=num_questions_per_doc,
                        document=doc)
                    try:
                        qa_list = FactQAGenerator.generate_question_for_fact(prompt, fact)
                        for qa in qa_list:
                            new_row = row.to_dict()
                            new_row['Question'] = qa.question
                            new_row['Answer'] = qa.answer
                            new_row['Fact'] = entity
                            print(entity)
                            data.append(new_row)
                    except:
                        pass
        except Exception as e:
                print(f"Failed to generate QA for document: {doc[:20]}. Error: {e:20}")

    if(len(data) == 0):
        return pd.DataFrame()

    result = pd.DataFrame(data)
    result = result.dropna(subset=['Document', 'Question', 'Answer'])
    result = result[(result['Document'] != '') & (result['Question'] != '') & (result['Answer'] != '')]
    return result

In [6]:
def update_df_with_lambda(df, column_name, lambda_function):
     # Check if the column does not exist in the DataFrame, if not, initialize it with NA
    if column_name not in df.columns:
        df[column_name] = pd.NA
    
    # Apply the lambda function only to rows where the column value is NA
    df[column_name] = df.apply(lambda row: lambda_function(row) if pd.isna(row[column_name]) else row[column_name], axis=1)
    return df

In [7]:
from answer_context_filter import calculate_bleu_score, calculate_rouge_score

def calculate_scores(df):
    # Calculate BLEU and ROUGE scores
    #df['Question-Context BLEU'] = df.apply(lambda row: calculate_bleu_score(row['Document'], row['Question']), axis=1)
    #df['Answer-Context BLEU'] = df.apply(lambda row: calculate_bleu_score(row['Document'], row['Answer']), axis=1)

    #df['Question-Context ROUGE'] = df.apply(lambda row: calculate_rouge_score(row['Document'], row['Question']), axis=1)
    #df['Answer-Context ROUGE'] = df.apply(lambda row: calculate_rouge_score(row['Document'], row['Answer']), axis=1)
    #df['Question-Context ROUGE-L F1'] = df['Question-Context ROUGE'].apply(lambda x: x[0]['rouge-l']['f'] if x else None)
    #df['Answer-Context ROUGE-L F1'] = df['Answer-Context ROUGE'].apply(lambda x: x[0]['rouge-l']['f'] if x else None)

    df = update_df_with_lambda(
        df, 'Question-Context ROUGE',
        lambda row: calculate_rouge_score(row['Document'], row['Question']) if pd.isna(row.get('Question-Context ROUGE')) else row['Question-Context ROUGE']
    )

    # Update 'Answer-Context ROUGE' only if it is NA
    df = update_df_with_lambda(
        df, 'Answer-Context ROUGE',
        lambda row: calculate_rouge_score(row['Document'], row['Answer']) if pd.isna(row.get('Answer-Context ROUGE')) else row['Answer-Context ROUGE']
    )

    # Update 'Question-Context ROUGE-L F1' based on the 'Question-Context ROUGE' column, only if it is NA
    df = update_df_with_lambda(
        df, 'Question-Context ROUGE-L F1',
        lambda row: row['Question-Context ROUGE'][0]['rouge-l']['f'] if pd.isna(row.get('Question-Context ROUGE-L F1')) and row['Question-Context ROUGE'] else None
    )

    # Update 'Answer-Context ROUGE-L F1' based on the 'Answer-Context ROUGE' column, only if it is NA
    df = update_df_with_lambda(
        df, 'Answer-Context ROUGE-L F1',
        lambda row: row['Answer-Context ROUGE'][0]['rouge-l']['f'] if pd.isna(row.get('Answer-Context ROUGE-L F1')) and row['Answer-Context ROUGE'] else None
    )

    return df


def rougeFilter(df):
    df = calculate_scores(df)
    df = df[df['Answer-Context ROUGE-L F1'] > 0.1]
    return df

## Vector filter

In [8]:
from vector_filter import precompute_embeddings
from vector_filter import filter_dataframe

def vectorFilter(df):
    df["Embedding"] = precompute_embeddings(df, 'Question')
    df = filter_dataframe(df, threshold=0.9)
    return df

## Entity Filter

In [9]:
def contains_all_elements(list1, list2):
    """
    Check if list1 contains all elements of list2.
    Each element is a dictionary with 'entityType' and 'value' keys.
    """
    # Convert each dictionary in the lists to a tuple (entityType, value) for easy comparison
    #set1 = {tuple(d.items()) for d in list1}
    #set2 = {tuple(d.items()) for d in list2}
    set1 = {d['value'] for d in list1 if 'value' in d}
    set2 = {d['value'] for d in list2 if 'value' in d}

    # Check if every element in set2 is also in set1
    return set2.issubset(set1)

def check_entity_match(row):
    return contains_all_elements(row['Context Entities'], row['Answer Entities'])

# Apply the function to each row to create the new column


In [10]:
from extract import extract_all_entities

def extractEntities(df):
    df['Context Entities'] = df['Document'].apply(extract_all_entities)
    df['Question Entities'] = df['Question'].apply(extract_all_entities)
    df['Answer Entities'] = df['Answer'].apply(extract_all_entities)
    df['Entity Match'] = df.apply(check_entity_match, axis=1)
    """  
    df = update_df_with_lambda(df, 'Context Entities', 
                               lambda row: extract_all_entities(row['Document']) 
                                if row.get('Context Entities') is None else row['Context Entities'])

    # Update or create 'Question Entities' column by applying entity extraction function if the value is NA
    df = update_df_with_lambda(df, 'Question Entities', 
                               lambda row: extract_all_entities(row['Question']) 
                               if row.get('Question Entities')is None else row['Question Entities'])

    # Update or create 'Answer Entities' column by applying entity extraction function if the value is NA
    df = update_df_with_lambda(df, 'Answer Entities', 
                               lambda row: extract_all_entities(row['Answer']) 
                               if row.get('Answer Entities') is None else row['Answer Entities'])

    # Update or create 'Entity Match' column by applying the check entity match function if the value is NA
    df = update_df_with_lambda(df, 'Entity Match', 
                               lambda row: check_entity_match(row) 
                               if row.get('Entity Match') is None else row['Entity Match'])
    """
    return df



/home/jovyan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-01-18 14:20:21.527565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 14:20:21.656910: I tensorflow/core/util/port.cc:104] oneDNN custom operations are 

2024-01-18 14:20:37,966 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-PER, S-LOC, B-MISC, E-MISC, B-ORG, E-ORG, I-ORG, I-PER, B-LOC, I-LOC, E-LOC, I-MISC, <START>, <STOP>


In [11]:
def contains_desired_entity(entities_list):
    desired_types = {'Phone', 'Email'}
    return any(entity['entityType'] in desired_types for entity in entities_list)


def filterFactQuestions(df):
    df = extractEntities(df)
    return df[~((df['Question'].str.split().str.len() < 30) & df['Answer Entities'].apply(contains_desired_entity))]

def extract_desired_entities(entities_list):
    # Define the desired entity types
    desired_types = {'Phone', 'Email'}
    
    # Extract entities that match the desired types
    matching_entities = [entity for entity in entities_list if entity['entityType'] in desired_types]
    
    return matching_entities

In [12]:
def filterFactQuestionCorrectness(df):
    df['Answer Entities'] = df['Answer'].apply(extract_all_entities)
    # Apply a filter condition to each row
    #filtered_df = df[df.apply(lambda row: row["Fact"] in row["Answer Entities"], axis=1)]
    filtered_df = df[df.apply(lambda row: any( row["Fact"]['value'] == entity['value'] for entity in row["Answer Entities"]) if isinstance(row["Answer Entities"], list) else False, axis=1)]
    return filtered_df

#### LLM Scoring

In [13]:
from llm_filter import estimate_relevance

def llmScore(df):
    return update_df_with_lambda(df, 'LLM Score', lambda row: estimate_relevance(row['Question'], row['Document']))

# Run

In [14]:
import os

sample = loadData(fname, basepath)

df = pd.DataFrame(columns = sample.columns)



for i in tqdm(range(5), desc="Processing"):
    raw_file_path = f"{basepath}/output_raw_{i}_{fname}.csv"

    if os.path.exists(raw_file_path):
        # Load the file into a DataFrame if it exists
        print(f"Skipping generation: {i}")
        new_questions_df = pd.read_csv(raw_file_path)
    else:
        df_gen = sample[~sample['Document'].isin(df['Document'])]
        new_questions_df = generateQuestions(df_gen)
        if df_gen.empty:
            break
    new_questions_df.to_csv(raw_file_path)
    df = pd.concat([df, new_questions_df], ignore_index=True, sort=False)
    df = rougeFilter(df)
    df = vectorFilter(df)
    df = filterFactQuestions(df)
    df = llmScore(df)


## take top one for each doc
idx = df.groupby('Document')['LLM Score'].idxmax()
df = df.loc[idx]

df.to_csv(f"{basepath}/output_full_{fname}.csv")
df.to_csv(f"{basepath}/output_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Question', 'Answer'])


Generating QAs:   4%|▎         | 7/200 [01:17<32:46, 10.19s/it]

Failed to generate QA for doc: 31.0: Acupunctuur  (Nocepta): Nacontrole: In overleg met u wordt een afspraak voor de volgende behan. Error: 'list' object has no attribute 'qa_list'



Generating QAs:  16%|█▌        | 31/200 [07:45<55:39, 19.76s/it]

Failed to generate QA for doc: 0.0: Cystolithotripsie: Wanneer neemt u contact op?: U neemt contact op bij: 
- Koorts hoger dan 38,. Error: 'list' object has no attribute 'qa_list'



Generating QAs:  22%|██▎       | 45/200 [11:33<44:45, 17.33s/it]

...


Generating QAs:  67%|██████▋   | 6/9 [01:02<00:23,  7.83s/it]

Failed to generate QA for doc: 7.0: Operatief verwijderen van de baarmoeder: Gevolgen van een baarmoederverwijdering: - Geen menstr. Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4127 tokens (2079 in your prompt; 2048 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}



Generating QAs:  75%|███████▌  | 6/8 [01:41<00:28, 14.08s/it]

Failed to generate QA for doc: 7.0: Operatief verwijderen van de baarmoeder: Gevolgen van een baarmoederverwijdering: - Geen menstr. Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4126 tokens (2078 in your prompt; 2048 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}



Processing: 100%|██████████| 5/5 [1:53:23<00:00, 1360.78s/it]


## Export Q&A Pairs

In [15]:
df.to_csv(f"{basepath}/output_full_{fname}.csv")
df.to_csv(f"{basepath}/output_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Question', 'Answer'])

In [16]:
df

,topic_id,filename,header,section_text,Document,Question,Answer,Question-Context ROUGE,Answer-Context ROUGE,Question-Context ROUGE-L F1,Answer-Context ROUGE-L F1,Embedding,Context Entities,Question Entities,Answer Entities,Entity Match,LLM Score
29,0.0,(Langdurig) gebroken vliezen,Contact opnemen,Wij vragen u contact op te nemen als: \n- Uw t...,0.0: (Langdurig) gebroken vliezen: Contact opn...,Moet ik contact opnemen als mijn temperatuur h...,"Ja, u moet contact opnemen als uw temperatuur ...","[{'rouge-1': {'r': 0.1206896551724138, 'p': 0....","[{'rouge-1': {'r': 0.1896551724137931, 'p': 0....",0.162162,0.213333,"[-0.004962361079971868, -0.041561738727993854,...",[],[],[],True,100.0
156,0.0,Aambeien (heamorroiden),Vragen,Deze folder is niet bedoeld als vervanging van...,0.0: Aambeien (heamorroiden): Vragen: Deze fol...,Waar kan ik terecht voor vragen over aambeien ...,U kunt op werkdagen bellen met het secretariaa...,"[{'rouge-1': {'r': 0.02531645569620253, 'p': 0...","[{'rouge-1': {'r': 0.189873417721519, 'p': 0.7...",0.044444,0.300000,"[-0.007706894348600569, -0.030702567838503225,...","[{'entityType': 'Phone', 'value': '+3188708523...",[],[],True,100.0
135,0.0,Anale fissuur,Vragen,Deze brochure is niet bedoeld als vervanging v...,0.0: Anale fissuur: Vragen: Deze brochure is n...,Kan deze brochure een mondelinge informatie ve...,"Nee, deze brochure is niet bedoeld als vervang...","[{'rouge-1': {'r': 0.045454545454545456, 'p': ...","[{'rouge-1': {'r': 0.13636363636363635, 'p': 0...",0.082192,0.233766,"[-0.021418902809556963, -0.008482350689619944,...","[{'entityType': 'Phone', 'value': '+3188708523...",[],[],True,100.0
557,0.0,Anale fissuur,Vragen,Deze brochure is niet bedoeld als vervanging v...,0.0: Anale fissuur: Vragen: Deze brochure is n...,Wanneer is het mogelijk om meer informatie te ...,U kunt meer informatie over de proctologiepoli...,"[{'rouge-1': {'r': 0.10126582278481013, 'p': 0...","[{'rouge-1': {'r': 0.08860759493670886, 'p': 0...",0.175824,0.155556,"[0.008837624754946205, -0.014453528011422483, ...","[{'entityType': 'Phone', 'value': '+3188708523...",[],[],True,100.0
177,0.0,"Angiografie, dotterbehandeling en_of stentplaa...",Neem contact op indien,"- U koorts krijgt (38,5°C of hoger). \n- De li...","0.0: Angiografie, dotterbehandeling en_of sten...",Wat moet ik doen als mijn lies rood verkleurt ...,Als uw lies rood verkleurt en erg warm aanvoel...,"[{'rouge-1': {'r': 0.13432835820895522, 'p': 0...","[{'rouge-1': {'r': 0.29850746268656714, 'p': 0...",0.209302,0.371134,"[-0.016909901298582258, -0.029433875324452668,...","[{'entityType': 'Phone', 'value': '+3188708524...",[],[],True,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,7.0,Serotiniteit,Wat is serotiniteit?,Als de bevalling twee weken na de uitgerekende...,7.0: Serotiniteit: Wat is serotiniteit?: Als d...,Wat is serotiniteit?,Serotiniteit is de medische term voor overdrag...,"[{'rouge-1': {'r': 0.05263157894736842, 'p': 0...","[{'rouge-1': {'r': 0.34210526315789475, 'p': 0...",0.097561,0.440678,"[0.019936117249290446, -0.024929900769007817, ...",[],[],[],True,100.0
17,8.0,"Borstvorming bij mannen _ gynaecomastie, behan...",Borstvorming bij mannen,Gynaecomastie Deze folder geeft u een globaal ...,"8.0: Borstvorming bij mannen _ gynaecomastie, ...",Wat is gynaecomastie?,Gynaecomastie is borstvorming bij mannen. Het ...,"[{'rouge-1': {'r': 0.021739130434782608, 'p': ...","[{'rouge-1': {'r': 0.1956521739130435, 'p': 0....",0.040816,0.262295,"[-0.023004339812470277, -0.028726620355583557,...",[],[],[],True,100.0
370,8.0,"LAR-Syndroom, vervolgbehandeling",Vervolgbehandeling Low Anterior Resectie-,Syndroom; “LAR-Syndroom” Na een endeldarmopera...,"8.0: LAR-Syndroom, vervolgbehandeling: Vervolg...",Wat is het LAR-Syndroom?,Het LAR-Syndroom is een verzameling van klacht...,"[{'rouge-1': {'r': 0.028985507246376812, 'p': ...","[{'rouge-1': {'r': 0.13043478260869565, 'p': 0...",0.054795,0.148148,"[0.00635609825801561, -0.01220764

# Fact Q&A

In [17]:
sample = loadData(fname, basepath)

sample['Context Entities'] = sample['Document'].apply(extract_all_entities)

new_questions_df = generateFactQuestions(sample)
new_questions_df.to_csv(f"{basepath}/output_fact_raw_{fname}.csv")
df = vectorFilter(new_questions_df)
df = llmScore(df)
df = filterFactQuestionCorrectness(df)

## take top one for each doc
df['Fact Value'] = df['Fact'].apply(lambda x: x.get('value'))
idx = df.groupby(['Document', 'Fact Value'])['LLM Score'].idxmax()
df = df.loc[idx]

Generating Fact QAs:   0%|          | 0/200 [00:00<?, ?it/s]

{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}


Generating Fact QAs:   4%|▍         | 9/200 [00:09<03:22,  1.06s/it]

{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Email', 'value': 'info@heupafwijkingen.nl'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083

Generating Fact QAs:   8%|▊         | 17/200 [00:27<05:19,  1.75s/it]

...

{'entityType': 'Phone', 'value': '+31887083620'}
{'entityType': 'Phone', 'value': '+31887083620'}
{'entityType': 'Phone', 'value': '+31887083620'}
{'entityType': 'Phone', 'value': '+31887083620'}
{'entityType': 'Phone', 'value': '+31887083620'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}


Generating Fact QAs:  68%|██████▊   | 136/200 [08:09<00:38,  1.66it/s]

{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887087010'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}
{'entityType': 'Phone', 'value': '+31887083220'}


Generating Fact QAs: 100%|██████████| 200/200 [08:23<00:00,  2.52s/it]

{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}
{'entityType': 'Email', 'value': 'dietisten@zgt.nl'}


In [18]:

df.to_csv(f"{basepath}/output_full_fact_{fname}.csv")
df.to_csv(f"{basepath}/output_fact_{fname}.csv", columns = ['topic_id', 'filename', 'header', 'section_text', 'Fact', 'Question', 'Answer'])